# Investigation: Missing Shortcut 31283 → 31282

This notebook investigates why the shortcut `31283 → 31282` exists in Spark but is missing from DuckDB partitioned output.

**Path**: `31283 → 31280 → 31276 → 31282`

In [11]:
import duckdb
import h3
import pandas as pd

con = duckdb.connect(':memory:')

# Register H3 UDFs
def h3_lca(c1, c2):
    if c1 == 0 or c2 == 0: return 0
    s1, s2 = h3.int_to_str(c1), h3.int_to_str(c2)
    m = min(h3.get_resolution(s1), h3.get_resolution(s2))
    for r in range(m, -1, -1):
        if h3.cell_to_parent(s1, r) == h3.cell_to_parent(s2, r):
            return h3.str_to_int(h3.cell_to_parent(s1, r))
    return 0

def h3_res(c):
    if c == 0: return -1
    return h3.get_resolution(h3.int_to_str(c))

def h3_parent(c, r):
    if c == 0 or r < 0: return 0
    s = h3.int_to_str(c)
    if h3.get_resolution(s) < r: return c
    return h3.str_to_int(h3.cell_to_parent(s, r))

con.create_function("h3_lca", h3_lca, ["BIGINT", "BIGINT"], "BIGINT")
con.create_function("h3_resolution", h3_res, ["BIGINT"], "INTEGER")
con.create_function("h3_parent", h3_parent, ["BIGINT", "INTEGER"], "BIGINT")

print("UDFs registered.")

UDFs registered.


## 1. Load Data

In [12]:
district = "Burnaby"
edges_path = f"/home/kaveh/projects/osm-to-road/data/output/{district}/{district}_driving_simplified_edges_with_h3.csv"
graph_path = f"/home/kaveh/projects/osm-to-road/data/output/{district}/{district}_driving_edge_graph.csv"
spark_output = f"/home/kaveh/projects/road-to-shortcut/output/{district}_spark_hybrid/*.parquet"
duck_output = f"/home/kaveh/projects/road-to-shortcut-duckdb/output/{district}_shortcuts"

con.execute(f"CREATE TABLE edges AS SELECT * FROM read_csv_auto('{edges_path}')")
con.execute(f"CREATE TABLE graph AS SELECT * FROM read_csv_auto('{graph_path}')")
con.execute(f"CREATE TABLE spark AS SELECT * FROM read_parquet('{spark_output}')")
con.execute(f"CREATE TABLE duck AS SELECT * FROM read_parquet('{duck_output}')")

print("Data loaded.")

Data loaded.


## 2. The Four Edges Involved

In [13]:
edge_ids = [31283, 31280, 31276, 31282]

edges_df = con.execute(f"""
    SELECT edge_index, length, maxspeed, highway, from_cell, to_cell, lca_res
    FROM edges WHERE edge_index IN {tuple(edge_ids)}
""").df()

display(edges_df)

,edge_index,length,maxspeed,highway,from_cell,to_cell,lca_res
0,31276,8.758,20.0,service,644733694798254108,644733694798254449,12
1,31280,47.126,20.0,service,644733727768520914,644733694798254108,3
2,31282,52.031,20.0,service,644733694798254449,644733727768520914,3
3,31283,2.121,20.0,service,644733727768518952,644733727768520914,11


## 3. The Three Shortcuts (Elementary)

In [14]:
shortcut_pairs = [(31283, 31280), (31280, 31276), (31276, 31282)]

for f, t in shortcut_pairs:
    print(f"\n=== Shortcut {f} → {t} ===")
    
    # Check if elementary (direct edge connection)
    is_elem = con.execute(f"SELECT count(*) FROM graph WHERE from_edge={f} AND to_edge={t}").fetchone()[0]
    print(f"  Elementary (direct): {'YES' if is_elem else 'NO'}")
    
    # Spark version
    spark_row = con.execute(f"SELECT * FROM spark WHERE from_edge={f} AND to_edge={t}").df()
    if not spark_row.empty:
        print(f"  Spark: cost={spark_row['cost'].values[0]:.6f}, via={spark_row['via_edge'].values[0]}")
    else:
        print("  Spark: MISSING")
    
    # DuckDB version
    duck_row = con.execute(f"SELECT * FROM duck WHERE from_edge={f} AND to_edge={t}").df()
    if not duck_row.empty:
        print(f"  DuckDB: cost={duck_row['cost'].values[0]:.6f}, via={duck_row['via_edge'].values[0]}")
    else:
        print("  DuckDB: MISSING")


=== Shortcut 31283 → 31280 ===
  Elementary (direct): YES
  Spark: cost=0.106050, via=31280
  DuckDB: cost=0.106050, via=31280

=== Shortcut 31280 → 31276 ===
  Elementary (direct): YES
  Spark: cost=2.356300, via=31276
  DuckDB: cost=2.356300, via=31276

=== Shortcut 31276 → 31282 ===
  Elementary (direct): YES
  Spark: cost=0.437900, via=31282
  DuckDB: cost=0.437900, via=31282


## 4. The Missing Compound Shortcut

In [15]:
print("=== Compound Shortcut 31283 → 31282 ===")

spark_row = con.execute("SELECT * FROM spark WHERE from_edge=31283 AND to_edge=31282").df()
duck_row = con.execute("SELECT * FROM duck WHERE from_edge=31283 AND to_edge=31282").df()

print(f"Spark: {spark_row.to_dict('records') if not spark_row.empty else 'MISSING'}")
print(f"DuckDB: {duck_row.to_dict('records') if not duck_row.empty else 'MISSING'}")

=== Compound Shortcut 31283 → 31282 ===
Spark: [{'from_edge': 31283, 'to_edge': 31282, 'cost': 2.90025, 'via_edge': 31280, 'inside': -2, 'cell': 590690563070623743}]
DuckDB: MISSING


## 5. Cell Assignment Analysis

For each shortcut, we calculate:
- `inner_cell`: LCA of (from_edge.to_cell, to_edge.from_cell)
- `outer_cell`: LCA of (from_edge.from_cell, to_edge.to_cell)
- `lca_res`: max(from_edge.lca_res, to_edge.lca_res)

In [16]:
analysis_pairs = [(31283, 31280), (31280, 31282), (31283, 31282)]

for f, t in analysis_pairs:
    print(f"\n=== Cell Analysis: {f} → {t} ===")
    
    result = con.execute(f"""
        SELECT 
            e1.edge_index as from_edge,
            e2.edge_index as to_edge,
            e1.from_cell as A_from,
            e1.to_cell as A_to,
            e2.from_cell as B_from,
            e2.to_cell as B_to,
            e1.lca_res as lca_in,
            e2.lca_res as lca_out,
            GREATEST(e1.lca_res, e2.lca_res) as lca_res,
            h3_lca(e1.to_cell, e2.from_cell) as inner_cell,
            h3_resolution(h3_lca(e1.to_cell, e2.from_cell)) as inner_res,
            h3_lca(e1.from_cell, e2.to_cell) as outer_cell,
            h3_resolution(h3_lca(e1.from_cell, e2.to_cell)) as outer_res
        FROM edges e1, edges e2
        WHERE e1.edge_index = {f} AND e2.edge_index = {t}
    """).df()
    
    row = result.iloc[0]
    print(f"  LCA Res (shortcut): {row['lca_res']}")
    print(f"  Inner Cell: {row['inner_cell']} (Res {row['inner_res']})")
    print(f"  Outer Cell: {row['outer_cell']} (Res {row['outer_res']})")
    print(f"  Valid at Res 11? lca_res({row['lca_res']}) <= 11 AND (inner_res({row['inner_res']}) >= 11 OR outer_res({row['outer_res']}) >= 11)")
    valid = row['lca_res'] <= 11 and (row['inner_res'] >= 11 or row['outer_res'] >= 11)
    print(f"  → {'YES' if valid else 'NO'}")


=== Cell Analysis: 31283 → 31280 ===
  LCA Res (shortcut): 11
  Inner Cell: 644733727768520914 (Res 15)
  Outer Cell: 590690563070623743 (Res 3)
  Valid at Res 11? lca_res(11) <= 11 AND (inner_res(15) >= 11 OR outer_res(3) >= 11)
  → YES

=== Cell Analysis: 31280 → 31282 ===
  LCA Res (shortcut): 3
  Inner Cell: 631222895916143103 (Res 12)
  Outer Cell: 644733727768520914 (Res 15)
  Valid at Res 11? lca_res(3) <= 11 AND (inner_res(12) >= 11 OR outer_res(15) >= 11)
  → YES

=== Cell Analysis: 31283 → 31282 ===
  LCA Res (shortcut): 11
  Inner Cell: 590690563070623743 (Res 3)
  Outer Cell: 626719329259040767 (Res 11)
  Valid at Res 11? lca_res(11) <= 11 AND (inner_res(3) >= 11 OR outer_res(11) >= 11)
  → YES


## 6. Cell Assignment at Resolution 11

Check what cell each shortcut would be assigned to at Resolution 11.

In [17]:
for f, t in analysis_pairs:
    print(f"\n=== Assignment at Res 11: {f} → {t} ===")
    
    result = con.execute(f"""
        WITH base AS (
            SELECT 
                GREATEST(e1.lca_res, e2.lca_res) as lca_res,
                h3_lca(e1.to_cell, e2.from_cell) as inner_cell,
                h3_lca(e1.from_cell, e2.to_cell) as outer_cell
            FROM edges e1, edges e2
            WHERE e1.edge_index = {f} AND e2.edge_index = {t}
        )
        SELECT 
            lca_res,
            h3_resolution(inner_cell) as inner_res,
            h3_resolution(outer_cell) as outer_res,
            CASE WHEN h3_resolution(inner_cell) >= 11 THEN h3_parent(inner_cell, 11) END as inner_cell_11,
            CASE WHEN h3_resolution(outer_cell) >= 11 THEN h3_parent(outer_cell, 11) END as outer_cell_11
        FROM base
    """).df()
    
    row = result.iloc[0]
    print(f"  Inner Cell at Res 11: {row['inner_cell_11']}")
    print(f"  Outer Cell at Res 11: {row['outer_cell_11']}")


=== Assignment at Res 11: 31283 → 31280 ===
  Inner Cell at Res 11: 626719329259040767
  Outer Cell at Res 11: <NA>

=== Assignment at Res 11: 31280 → 31282 ===
  Inner Cell at Res 11: 626719296288776191
  Outer Cell at Res 11: 626719329259040767

=== Assignment at Res 11: 31283 → 31282 ===
  Inner Cell at Res 11: <NA>
  Outer Cell at Res 11: 626719329259040767


## 7. The Bug: Phase 1 Chunk Filtering

Check if both shortcuts would be loaded into the same Phase 1 chunk.

In [18]:
partition_res = 7

for f, t in [(31283, 31280), (31280, 31282)]:
    print(f"\n=== Phase 1 Chunk for {f} → {t} ===")
    
    result = con.execute(f"""
        WITH base AS (
            SELECT 
                h3_lca(e1.to_cell, e2.from_cell) as inner_cell,
                h3_lca(e1.from_cell, e2.to_cell) as outer_cell
            FROM edges e1, edges e2
            WHERE e1.edge_index = {f} AND e2.edge_index = {t}
        )
        SELECT 
            inner_cell,
            h3_resolution(inner_cell) as inner_res,
            h3_parent(inner_cell, {partition_res}) as inner_chunk,
            outer_cell,
            h3_resolution(outer_cell) as outer_res,
            h3_parent(outer_cell, {partition_res}) as outer_chunk
        FROM base
    """).df()
    
    row = result.iloc[0]
    print(f"  Inner: Res {row['inner_res']} → Chunk {row['inner_chunk']}")
    print(f"  Outer: Res {row['outer_res']} → Chunk {row['outer_chunk']}")
    
    # The bug: if inner_res < partition_res, h3_parent returns the original cell (not a valid chunk ID)
    if row['inner_res'] < partition_res:
        print(f"  ⚠️ BUG: inner_res ({row['inner_res']}) < partition_res ({partition_res}), inner_chunk is INVALID")


=== Phase 1 Chunk for 31283 → 31280 ===
  Inner: Res 15 → Chunk 608704930760359935
  Outer: Res 3 → Chunk 590690563070623743

=== Phase 1 Chunk for 31280 → 31282 ===
  Inner: Res 12 → Chunk 608704897793130495
  Outer: Res 15 → Chunk 608704930760359935


## 8. Visualization (Optional)

In [19]:
# Get geometries for the edges
geom_df = con.execute("""
    SELECT edge_index, geometry FROM edges WHERE edge_index IN (31283, 31280, 31276, 31282)
""").df()

print("Edge Geometries (WKT):")
for _, row in geom_df.iterrows():
    print(f"  {row['edge_index']}: {row['geometry'][:80]}...")

Edge Geometries (WKT):
  31276: LINESTRING (-123.02251434326172 49.26738357543945, -123.02259063720703 49.267322...
  31280: LINESTRING (-123.02313232421875 49.267433166503906, -123.02262115478516 49.26744...
  31282: LINESTRING (-123.02259063720703 49.2673225402832, -123.02313232421875 49.2673187...
  31283: LINESTRING (-123.02313232421875 49.267433166503906, -123.02313232421875 49.26745...


## 9. Birth Cell and Birth Resolution in Spark

The 'birth' of a shortcut is the first resolution at which it could be calculated.
- **Birth Resolution**: The finest resolution where both component shortcuts are active in the same cell.
- **Birth Cell**: The H3 cell where the shortcut was first computed.

In [20]:
# For shortcuts, the 'birth resolution' is when both components first meet
# This is determined by max(inner_res, outer_res) for each shortcut

shortcuts_to_analyze = [
    (31283, 31280, 'Elementary'),
    (31280, 31276, 'Elementary'),
    (31276, 31282, 'Elementary'),
    (31280, 31282, 'Compound (2-hop)'),
    (31283, 31282, 'Compound (3-hop, MISSING in DuckDB)')
]

print('='*80)
print(f"{'Shortcut':<20} {'Type':<30} {'Birth Res':>10} {'Birth Cell':>20}")
print('='*80)

for f, t, desc in shortcuts_to_analyze:
    result = con.execute(f"""
        WITH base AS (
            SELECT 
                GREATEST(e1.lca_res, e2.lca_res) as lca_res,
                h3_lca(e1.to_cell, e2.from_cell) as inner_cell,
                h3_lca(e1.from_cell, e2.to_cell) as outer_cell,
                h3_resolution(h3_lca(e1.to_cell, e2.from_cell)) as inner_res,
                h3_resolution(h3_lca(e1.from_cell, e2.to_cell)) as outer_res
            FROM edges e1, edges e2
            WHERE e1.edge_index = {f} AND e2.edge_index = {t}
        )
        SELECT 
            lca_res,
            inner_cell, inner_res,
            outer_cell, outer_res,
            GREATEST(inner_res, outer_res) as birth_res,
            CASE 
                WHEN inner_res >= outer_res THEN inner_cell
                ELSE outer_cell
            END as birth_cell
        FROM base
    """).df()
    
    row = result.iloc[0]
    print(f"{f} → {t:<10} {desc:<30} {row['birth_res']:>10} {row['birth_cell']:>20}")

Shortcut             Type                            Birth Res           Birth Cell
31283 → 31280      Elementary                             15   644733727768520914
31280 → 31276      Elementary                             15   644733694798254108
31276 → 31282      Elementary                             15   644733694798254449
31280 → 31282      Compound (2-hop)                       15   644733727768520914
31283 → 31282      Compound (3-hop, MISSING in DuckDB)         11   626719329259040767


In [21]:
# Detailed birth analysis for the key shortcuts
print('\n' + '='*80)
print('DETAILED BIRTH ANALYSIS')
print('='*80)

for f, t in [(31283, 31280), (31280, 31282), (31283, 31282)]:
    print(f"\n--- Shortcut {f} → {t} ---")
    
    result = con.execute(f"""
        WITH base AS (
            SELECT 
                GREATEST(e1.lca_res, e2.lca_res) as lca_res,
                h3_lca(e1.to_cell, e2.from_cell) as inner_cell,
                h3_lca(e1.from_cell, e2.to_cell) as outer_cell,
                h3_resolution(h3_lca(e1.to_cell, e2.from_cell)) as inner_res,
                h3_resolution(h3_lca(e1.from_cell, e2.to_cell)) as outer_res
            FROM edges e1, edges e2
            WHERE e1.edge_index = {f} AND e2.edge_index = {t}
        )
        SELECT *,
            GREATEST(inner_res, outer_res) as birth_res,
            CASE 
                WHEN inner_res >= outer_res THEN inner_cell
                ELSE outer_cell
            END as birth_cell,
            CASE 
                WHEN inner_res >= outer_res THEN 'INNER'
                ELSE 'OUTER'
            END as birth_type
        FROM base
    """).df()
    
    row = result.iloc[0]
    print(f"  LCA Res: {row['lca_res']}")
    print(f"  Inner Cell: {row['inner_cell']} (Res {row['inner_res']})")
    print(f"  Outer Cell: {row['outer_cell']} (Res {row['outer_res']})")
    print(f"  → Birth Resolution: {row['birth_res']}")
    print(f"  → Birth Cell: {row['birth_cell']} (via {row['birth_type']})")
    
    # Show the H3 cell in hex format for easier reading
    birth_cell_hex = h3.int_to_str(int(row['birth_cell']))
    print(f"  → Birth Cell (Hex): {birth_cell_hex}")


DETAILED BIRTH ANALYSIS

--- Shortcut 31283 → 31280 ---
  LCA Res: 11
  Inner Cell: 644733727768520914 (Res 15)
  Outer Cell: 590690563070623743 (Res 3)
  → Birth Resolution: 15
  → Birth Cell: 644733727768520914 (via INNER)
  → Birth Cell (Hex): 8f28de8d25b28d2

--- Shortcut 31280 → 31282 ---
  LCA Res: 3
  Inner Cell: 631222895916143103 (Res 12)
  Outer Cell: 644733727768520914 (Res 15)
  → Birth Resolution: 15
  → Birth Cell: 644733727768520914 (via OUTER)
  → Birth Cell (Hex): 8f28de8d25b28d2

--- Shortcut 31283 → 31282 ---
  LCA Res: 11
  Inner Cell: 590690563070623743 (Res 3)
  Outer Cell: 626719329259040767 (Res 11)
  → Birth Resolution: 11
  → Birth Cell: 626719329259040767 (via OUTER)
  → Birth Cell (Hex): 8b28de8d25b2fff


In [ ]:
# Check if shortcuts 31283->31280 and 31280->31282 would meet at resolution 11
print('\n' + '='*80)
print('CAN THESE SHORTCUTS MEET AT RESOLUTION 11?')
print('='*80)

# Get the cells each shortcut would be assigned to at res 11
for f, t in [(31283, 31280), (31280, 31282)]:
    result = con.execute(f"""
        WITH base AS (
            SELECT 
                GREATEST(e1.lca_res, e2.lca_res) as lca_res,
                h3_lca(e1.to_cell, e2.from_cell) as inner_cell,
                h3_lca(e1.from_cell, e2.to_cell) as outer_cell
            FROM edges e1, edges e2
            WHERE e1.edge_index = {f} AND e2.edge_index = {t}
        )
        SELECT 
            lca_res,
            h3_resolution(inner_cell) as inner_res,
            h3_resolution(outer_cell) as outer_res,
            CASE WHEN h3_resolution(inner_cell) >= 11 THEN h3_parent(inner_cell, 11) END as inner_11,
            CASE WHEN h3_resolution(outer_cell) >= 11 THEN h3_parent(outer_cell, 11) END as outer_11
        FROM base
    """).df()
    
    row = result.iloc[0]
    print(f"\nShortcut {f} → {t}:")
    print(f"  LCA Res: {row['lca_res']} (active at res 11: {row['lca_res'] <= 11})")
    print(f"  Can assign to Inner Cell at Res 11: {row['inner_11']} (valid: {pd.notna(row['inner_11'])})")
    print(f"  Can assign to Outer Cell at Res 11: {row['outer_11']} (valid: {pd.notna(row['outer_11'])})")


CAN THESE SHORTCUTS MEET AT RESOLUTION 11?

Shortcut 31283 → 31280:
  LCA Res: 11 (active at res 11: True)
  Can assign to Inner Cell at Res 11: 626719329259040767 (valid: True)
  Can assign to Outer Cell at Res 11: <NA> (valid: False)

Shortcut 31280 → 31282:
  LCA Res: 3 (active at res 11: True)
  Can assign to Inner Cell at Res 11: 626719296288776191 (valid: True)
  Can assign to Outer Cell at Res 11: 626719329259040767 (valid: True)


: 